In [1]:
# this shall train neural network to recognize category / words ( total 30 ) from 1 second audio
# input audio format : format 16-bit little-endian PCM-encoded WAVE file at a 16000 sample rate

# to do : deal with padding and batching , deal with audio process / dataset , loss function

import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import scipy.io.wavfile as wav
from python_speech_features import mfcc
import numpy as np


In [2]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
    #d_mfcc_feat = delta(mfcc_feat, 2)
    #fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [3]:
# test above function
inputs = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [14]:
# function to convert audio category / label to unique number


   

def label_to_number(category):

    
    word_list = ['bed','bird','cat','dog','down','eight','five','four','go','happy','house','left','marvin','nice','no','off','on','one','right','seven','sheila','six','stop','three','tree','two','up','wow','yes','zero','_background_noise_']
    word_dictionary = {k: v for v, k in enumerate(word_list)}
    
  #  print (word_dictionary)
    
    for k, v in word_dictionary.items():
        if k == category:
            return v
    

    
    return -1
    
    
     
    


In [15]:
print(label_to_number('cat'))

2


In [6]:
# Training Parameters
learning_rate = 0.001
training_steps = 20
batch_size = 2
display_step = 200


# Network Parameters

num_features = 13

timesteps = 300 # maximum timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 30   # since there are 30 categories 

In [7]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_features])


In [8]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([batch_size, 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [9]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    # get ony the output of last rnn timestep
   # outputs=outputs[-1]

    outputs=tf.transpose( outputs , [1, 0, 2])
    
    
    
    
    
  
   # res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    return tf.nn.softmax(tf.matmul(outputs, weights['out'])[-1][-1] + biases['out'])

In [10]:
inputs = tf.placeholder(tf.float32, [None, None, num_features])
y = tf.placeholder("int32")

temp = tf.one_hot(y,30)




In [11]:
logits = RNN(inputs, weights, biases)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=temp))

In [12]:
init = tf.global_variables_initializer()

In [13]:
with tf.Session() as sess:
    

    # Run the initializer
    sess.run(init)
    
    feed = { inputs : np.random.rand(2, 300 , 13) , y : 1 }
    
    #print(sess.run(logits,feed  ))
    
    print(sess.run(loss_op, feed))

3.43436
